In [1]:
import tensorflow as tf
from gene_expression import *
from pathway_hierarchy import *
from utils import *
import torch

2024-07-24 17:47:42.072941: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-24 17:47:42.082877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 17:47:42.094714: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 17:47:42.094735: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 17:47:42.104152: I tensorflow/core/platform/cpu_feature_gua

In [2]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-07-24 17:47:43.733025: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.733712: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.733822: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
import pandas as pd
import numpy as np

In [4]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2024-07-24 17:47:43.754559: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.754677: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.754777: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

True

L355
2024-07-24 17:47:43.915440: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.915538: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-24 17:47:43.915623: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /device:GPU:0 with 43618 MB memory:  -> device: 0, name: NVIDIA RTX 6000 Ada Generation, pci bus id: 0000:01:00.0, compute capability: 8.9


In [5]:
torch.cuda.is_available()


True

In [6]:
from biomart import BiomartServer

# Connect to the Ensembl Biomart server
server = BiomartServer("http://www.ensembl.org/biomart")

# Access the Ensembl Genes dataset for Homo sapiens
dataset = server.datasets['hsapiens_gene_ensembl']

# Query the dataset for Ensembl IDs and gene names
response = dataset.search({
    'attributes': [
        'ensembl_gene_id', 
        'external_gene_name'
    ]
})

# Convert the response to a dataframe
import pandas as pd
from io import StringIO

data_ensemble_gene = StringIO(response.text)

df_ensemble = pd.read_csv(data_ensemble_gene , sep="\t",header = None)

# Display the first few rows of the dataframe
print(df_ensemble.head())
# Nice

                 0        1
0  ENSG00000210049    MT-TF
1  ENSG00000211459  MT-RNR1
2  ENSG00000210077    MT-TV
3  ENSG00000210082  MT-RNR2
4  ENSG00000209082   MT-TL1


In [7]:
import numpy as np
def return_ensemble_id(x):
    try:
        return df_ensemble[df_ensemble[1]==x][0].values[0]
    except: 
        return np.nan


# Excitory_neurons_1


In [8]:
import pickle
import os
# Load the DataFrame from the pickle file
#data = pd.read_csv('../preprocessed_data/mat.csv')
l = 0
for i in os.listdir('../../usman/Single_Cell_Microglia_Project/preprocessed_data/excitory_neurons/split_chunks/excitory_neurons_1/'):
    try:
        if i.split('.')[1] == 'pkl':
            print(i)
            pickle_file_path = '../../usman/Single_Cell_Microglia_Project/preprocessed_data/excitory_neurons/split_chunks/excitory_neurons_1/' + i
            if l == 0: 
                data = pd.read_pickle(pickle_file_path)
                # to load data in the RAM (not possible to load all data once)
                selected_columns = np.random.choice(data.columns, size=int(len(data.columns) *0.2), replace=False)
                data = d
                ata[selected_columns]
            else:
                df = pd.read_pickle(pickle_file_path)
                df_selected = df[selected_columns] # selected columns comimg from first time loaded data
                data = pd.concat([data,df_selected],axis = 0)
            
            print("Pickle file has been loaded into a DataFrame:")
            l = l + 1       
    except:
        continue

excitory_neurons_mat_chunk_16.pkl
excitory_neurons_mat_chunk_14.pkl
excitory_neurons_mat_chunk_2.pkl
excitory_neurons_mat_chunk_9.pkl
excitory_neurons_mat_chunk_8.pkl
excitory_neurons_mat_chunk_5.pkl
excitory_neurons_mat_chunk_7.pkl
excitory_neurons_mat_chunk_10.pkl
excitory_neurons_mat_chunk_4.pkl
excitory_neurons_mat_chunk_12.pkl
excitory_neurons_mat_chunk_3.pkl
excitory_neurons_mat_chunk_1.pkl
excitory_neurons_mat_chunk_17.pkl
excitory_neurons_mat_chunk_15.pkl
excitory_neurons_mat_chunk_6.pkl
excitory_neurons_mat_chunk_11.pkl
excitory_neurons_mat_chunk_13.pkl


In [9]:
data

,AACGTTGGTTCAGGCC.1.0,AAGACCTAGTTAACGA.1.0,AAGTCTGTCCTCAATT.1.0,AATCCAGAGTATGACA.1.0,AATCCAGTCCTTTCTC.1.0,ACACCAAAGCTCCCAG.1.0,ACATCAGCAGGCTCAC.1.0,ACCTTTACAATAACGA.1.0,ACGAGCCGTGCAGACA.1.0,ACGGCCAAGGAGTTGC.1.0,...,TTGATGGTCTCCCATG.16.14,TTGCATTTCCCATGGG.16.14,TTGCCTGAGAGTGTGC.16.14,TTGCCTGCATGTGACT.16.14,TTGGATGAGCTTCATG.16.14,TTGGGATTCTTCACAT.16.14,TTTCACATCCCTTTGG.16.14,TTTCACATCCGCAGTG.16.14,TTTGGTTCACTCCACT.16.14,TTTGTTGAGCCTCCAG.16.14
AC087463.4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
AC087463.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC087463.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PWRN3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
PWRN1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,8,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AC007336.2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MMP2-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LPCAT2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
AC007336.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:

x = data.reset_index()['index'].apply(return_ensemble_id)

In [11]:
x.value_counts()


index
ENSG00000256812    1
ENSG00000259905    1
ENSG00000185823    1
ENSG00000128739    1
ENSG00000273173    1
                  ..
ENSG00000275917    1
ENSG00000186399    1
ENSG00000282773    1
ENSG00000261794    1
ENSG00000274734    1
Name: count, Length: 1175, dtype: int64

In [12]:
data['ensemble_gene_name'] = x.tolist()

In [13]:
data
data = data.dropna()
data

,AACGTTGGTTCAGGCC.1.0,AAGACCTAGTTAACGA.1.0,AAGTCTGTCCTCAATT.1.0,AATCCAGAGTATGACA.1.0,AATCCAGTCCTTTCTC.1.0,ACACCAAAGCTCCCAG.1.0,ACATCAGCAGGCTCAC.1.0,ACCTTTACAATAACGA.1.0,ACGAGCCGTGCAGACA.1.0,ACGGCCAAGGAGTTGC.1.0,...,TTGCATTTCCCATGGG.16.14,TTGCCTGAGAGTGTGC.16.14,TTGCCTGCATGTGACT.16.14,TTGGATGAGCTTCATG.16.14,TTGGGATTCTTCACAT.16.14,TTTCACATCCCTTTGG.16.14,TTTCACATCCGCAGTG.16.14,TTTGGTTCACTCCACT.16.14,TTTGTTGAGCCTCCAG.16.14,ensemble_gene_name
PWRN1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,8,0,0,ENSG00000259905
NPAP1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000185823
SNRPN,3,0,3,0,10,4,0,0,0,4,...,0,0,4,3,0,0,17,0,6,ENSG00000128739
SNURF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000273173
UBE3A,1,0,1,1,5,4,4,2,0,1,...,9,1,1,8,7,2,6,10,2,ENSG00000114062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IRX6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000159387
MMP2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000087245
MMP2-AS1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000260135
LPCAT2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,ENSG00000087253


In [14]:
data.set_index('ensemble_gene_name', inplace = True)

In [15]:
data

,AACGTTGGTTCAGGCC.1.0,AAGACCTAGTTAACGA.1.0,AAGTCTGTCCTCAATT.1.0,AATCCAGAGTATGACA.1.0,AATCCAGTCCTTTCTC.1.0,ACACCAAAGCTCCCAG.1.0,ACATCAGCAGGCTCAC.1.0,ACCTTTACAATAACGA.1.0,ACGAGCCGTGCAGACA.1.0,ACGGCCAAGGAGTTGC.1.0,...,TTGATGGTCTCCCATG.16.14,TTGCATTTCCCATGGG.16.14,TTGCCTGAGAGTGTGC.16.14,TTGCCTGCATGTGACT.16.14,TTGGATGAGCTTCATG.16.14,TTGGGATTCTTCACAT.16.14,TTTCACATCCCTTTGG.16.14,TTTCACATCCGCAGTG.16.14,TTTGGTTCACTCCACT.16.14,TTTGTTGAGCCTCCAG.16.14
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000259905,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,8,0,0
ENSG00000185823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000128739,3,0,3,0,10,4,0,0,0,4,...,0,0,0,4,3,0,0,17,0,6
ENSG00000273173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000114062,1,0,1,1,5,4,4,2,0,1,...,0,9,1,1,8,7,2,6,10,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000159387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000087245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000260135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
data = data.T
data

ensemble_gene_name,ENSG00000259905,ENSG00000185823,ENSG00000128739,ENSG00000273173,ENSG00000114062,ENSG00000235731,ENSG00000206190,ENSG00000206187,ENSG00000259150,ENSG00000235160,...,ENSG00000177508,ENSG00000261803,ENSG00000283304,ENSG00000245694,ENSG00000176842,ENSG00000159387,ENSG00000087245,ENSG00000260135,ENSG00000087253,ENSG00000256812
AACGTTGGTTCAGGCC.1.0,1,0,3,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAGACCTAGTTAACGA.1.0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAGTCTGTCCTCAATT.1.0,0,0,3,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AATCCAGAGTATGACA.1.0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
AATCCAGTCCTTTCTC.1.0,0,0,10,0,5,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTGGGATTCTTCACAT.16.14,0,0,0,0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTCACATCCCTTTGG.16.14,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTCACATCCGCAGTG.16.14,8,0,17,0,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
TTTGGTTCACTCCACT.16.14,0,0,0,0,10,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
data = data.sort_index()
data

ensemble_gene_name,ENSG00000259905,ENSG00000185823,ENSG00000128739,ENSG00000273173,ENSG00000114062,ENSG00000235731,ENSG00000206190,ENSG00000206187,ENSG00000259150,ENSG00000235160,...,ENSG00000177508,ENSG00000261803,ENSG00000283304,ENSG00000245694,ENSG00000176842,ENSG00000159387,ENSG00000087245,ENSG00000260135,ENSG00000087253,ENSG00000256812
AAACCCAAGAAAGTCT.10.2,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAACC.15.10,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAGTC.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATCGCG.30.2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGACAGCTG.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTTAGCCC.13.6,0,0,0,0,6,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTCCAGC.9.13,0,0,1,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTCTGGC.16.9,1,0,0,0,10,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTGATTC.28.6,0,0,12,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
clinical_data = pd.read_csv('../../danish/preprocessed_data/clinical/clinical_single_cell.csv')
clinical_data

,individualID,individualIdSource,sex,ethnicity,pmi_df2,subject,projid,Study,msex,educ,...,age_death,cts_mmse30_first_ad_dx,cts_mmse30_lv,braaksc,ceradsc,cogdx,dcfdx_lv,clinical_diagnosis,pathological_diagnosis,clinical_pathological_AD
0,R2626559,Rush,male,NaN,6.500000,ROSMAP-45310,1211411,ROS,1.0,12.0,...,85.549624,NaN,24.0,1.0,4.0,4.0,4.0,AD,No AD,False
1,R9936070,Rush,male,NaN,7.016667,ROSMAP-34387,2899847,MAP,1.0,14.0,...,74.450376,NaN,27.0,2.0,2.0,3.0,3.0,False,False,False
2,R2367199,Rush,male,NaN,4.333333,ROSMAP-69520,3713990,MAP,1.0,12.0,...,87.928816,NaN,30.0,4.0,2.0,1.0,1.0,NCI,False,False
3,R9891381,Rush,female,NaN,6.916667,ROSMAP-53306,3889845,MAP,0.0,13.0,...,90.000000,NaN,22.0,2.0,1.0,2.0,2.0,False,AD,False
4,R9033345,Rush,male,NaN,4.166667,ROSMAP-79585,6107196,MAP,1.0,15.0,...,90.000000,NaN,22.0,5.0,1.0,4.0,4.0,AD,AD,AD_with_Plaques
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509,R9738414,Rush,female,NaN,9.250000,ROSMAP-90149,20976799,ROS,0.0,18.0,...,90.000000,NaN,19.0,2.0,4.0,2.0,2.0,False,No AD,False
510,R7737688,Rush,female,NaN,7.000000,NaN,69866926,MAP,0.0,13.0,...,84.104038,20.0,3.0,6.0,1.0,4.0,4.0,AD,AD,AD_with_Plaques
511,R7844746,Rush,male,NaN,0.866667,NaN,29286432,ROS,1.0,24.0,...,90.000000,NaN,29.0,5.0,2.0,2.0,2.0,False,False,False
512,R6692433,Rush,female,NaN,6.583333,NaN,48331728,MAP,0.0,14.0,...,82.715948,NaN,27.0,2.0,4.0,1.0,1.0,NCI,No AD,NCI_with_No_Plaques


In [19]:
meta_data = pd.read_csv('../../usman/Single_Cell_Microglia_Project/preprocessed_data/excitory_neurons/full_metadata_excitory_neurons_1_rds.csv',index_col = 0)
meta_data = meta_data.sort_values(by = ['cell_id'])
meta_data

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD
1,AAACCCAAGAAAGTCT-10-2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes
2,AAACCCAAGAATAACC-15-10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no
3,AAACCCAAGAATAGTC-32-11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no
4,AAACCCAAGAATCGCG-30-2,ROSMAP,7769,2888,4.801133,0.193075,0.889530,Exc L2-3 CBLN2 LINC02306,ROSMAP-42915,yes
5,AAACCCAAGACAGCTG-32-11,ROSMAP,5767,2795,0.104040,0.294781,0.916359,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no
...,...,...,...,...,...,...,...,...,...,...
296932,TTTGTTGTCTTAGCCC-13-6,ROSMAP,60432,8891,1.004435,0.314403,0.825921,Exc L2-3 CBLN2 LINC02306,ROSMAP-69123,yes
296933,TTTGTTGTCTTCCAGC-9-13,ROSMAP,14726,4732,1.052560,0.407443,0.881711,Exc L2-3 CBLN2 LINC02306,ROSMAP-57699,no
296934,TTTGTTGTCTTCTGGC-16-9,ROSMAP,16805,5654,1.684023,0.541506,0.888039,Exc L2-3 CBLN2 LINC02306,ROSMAP-70693,no
296935,TTTGTTGTCTTGATTC-28-6,ROSMAP,28745,7070,3.391894,0.424422,0.863377,Exc L2-3 CBLN2 LINC02306,ROSMAP-38931,no


In [20]:
meta_data = pd.merge(meta_data,clinical_data[['subject','clinical_pathological_AD']], on = 'subject', how = 'left')

meta_data

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,clinical_pathological_AD
0,AAACCCAAGAAAGTCT-10-2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
1,AAACCCAAGAAAGTCT-10-2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
2,AAACCCAAGAATAACC-15-10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no,NCI_with_No_Plaques
3,AAACCCAAGAATAGTC-32-11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
4,AAACCCAAGAATAGTC-32-11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...
391898,TTTGTTGTCTTAGCCC-13-6,ROSMAP,60432,8891,1.004435,0.314403,0.825921,Exc L2-3 CBLN2 LINC02306,ROSMAP-69123,yes,AD_with_Plaques
391899,TTTGTTGTCTTCCAGC-9-13,ROSMAP,14726,4732,1.052560,0.407443,0.881711,Exc L2-3 CBLN2 LINC02306,ROSMAP-57699,no,False
391900,TTTGTTGTCTTCTGGC-16-9,ROSMAP,16805,5654,1.684023,0.541506,0.888039,Exc L2-3 CBLN2 LINC02306,ROSMAP-70693,no,False
391901,TTTGTTGTCTTGATTC-28-6,ROSMAP,28745,7070,3.391894,0.424422,0.863377,Exc L2-3 CBLN2 LINC02306,ROSMAP-38931,no,NCI_with_No_Plaques


In [21]:
meta_data = meta_data[meta_data.clinical_pathological_AD != 'False']
meta_data

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,clinical_pathological_AD
0,AAACCCAAGAAAGTCT-10-2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
1,AAACCCAAGAAAGTCT-10-2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
2,AAACCCAAGAATAACC-15-10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no,NCI_with_No_Plaques
3,AAACCCAAGAATAGTC-32-11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
4,AAACCCAAGAATAGTC-32-11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...
391889,TTTGTTGTCTCTCGCA-7-4,ROSMAP,32152,6608,0.127519,0.276810,0.847547,Exc L2-3 CBLN2 LINC02306,ROSMAP-78346,yes,AD_with_Plaques
391893,TTTGTTGTCTGGACTA-8-7,ROSMAP,23007,5406,2.768723,0.269483,0.855799,Exc L2-3 CBLN2 LINC02306,ROSMAP-28143,no,NCI_with_No_Plaques
391894,TTTGTTGTCTGGGCCA-12-9,ROSMAP,14844,4598,2.021019,0.350310,0.877987,Exc L2-3 CBLN2 LINC02306,ROSMAP-65967,yes,AD_with_Plaques
391898,TTTGTTGTCTTAGCCC-13-6,ROSMAP,60432,8891,1.004435,0.314403,0.825921,Exc L2-3 CBLN2 LINC02306,ROSMAP-69123,yes,AD_with_Plaques


In [22]:
meta_data.clinical_pathological_AD.value_counts()

clinical_pathological_AD
NCI_with_No_Plaques    86515
AD_with_Plaques        58523
Name: count, dtype: int64

In [23]:
try:
    meta_data['cell_id'] = meta_data['cell_id'].str.replace('-', '.')
except:
    print('probably not needed to do')
meta_data.head()

/tmp/ipykernel_1308073/3567006923.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_data['cell_id'] = meta_data['cell_id'].str.replace('-', '.')


,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,clinical_pathological_AD
0,AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
1,AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
2,AAACCCAAGAATAACC.15.10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no,NCI_with_No_Plaques
3,AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
4,AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques


In [24]:
data

ensemble_gene_name,ENSG00000259905,ENSG00000185823,ENSG00000128739,ENSG00000273173,ENSG00000114062,ENSG00000235731,ENSG00000206190,ENSG00000206187,ENSG00000259150,ENSG00000235160,...,ENSG00000177508,ENSG00000261803,ENSG00000283304,ENSG00000245694,ENSG00000176842,ENSG00000159387,ENSG00000087245,ENSG00000260135,ENSG00000087253,ENSG00000256812
AAACCCAAGAAAGTCT.10.2,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAACC.15.10,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAGTC.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATCGCG.30.2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGACAGCTG.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTTAGCCC.13.6,0,0,0,0,6,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTCCAGC.9.13,0,0,1,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTCTGGC.16.9,1,0,0,0,10,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTTGATTC.28.6,0,0,12,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
meta_data = meta_data[meta_data.cell_id.isin(data.index.tolist())]
meta_data

,cell_id,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,clinical_pathological_AD
0,AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
1,AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
2,AAACCCAAGAATAACC.15.10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no,NCI_with_No_Plaques
3,AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
4,AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...,...
391889,TTTGTTGTCTCTCGCA.7.4,ROSMAP,32152,6608,0.127519,0.276810,0.847547,Exc L2-3 CBLN2 LINC02306,ROSMAP-78346,yes,AD_with_Plaques
391893,TTTGTTGTCTGGACTA.8.7,ROSMAP,23007,5406,2.768723,0.269483,0.855799,Exc L2-3 CBLN2 LINC02306,ROSMAP-28143,no,NCI_with_No_Plaques
391894,TTTGTTGTCTGGGCCA.12.9,ROSMAP,14844,4598,2.021019,0.350310,0.877987,Exc L2-3 CBLN2 LINC02306,ROSMAP-65967,yes,AD_with_Plaques
391898,TTTGTTGTCTTAGCCC.13.6,ROSMAP,60432,8891,1.004435,0.314403,0.825921,Exc L2-3 CBLN2 LINC02306,ROSMAP-69123,yes,AD_with_Plaques


In [26]:
meta_data.clinical_pathological_AD.value_counts()

clinical_pathological_AD
NCI_with_No_Plaques    86515
AD_with_Plaques        58523
Name: count, dtype: int64

In [27]:
meta_data_NCI = meta_data[meta_data.clinical_pathological_AD == 'NCI_with_No_Plaques'][0:min(meta_data.clinical_pathological_AD.value_counts().tolist())]
meta_data_AD = meta_data[meta_data.clinical_pathological_AD == 'AD_with_Plaques'][0:min(meta_data.clinical_pathological_AD.value_counts().tolist())]

In [28]:
meta_data = pd.concat([meta_data_NCI,meta_data_AD])
meta_data.set_index('cell_id',inplace = True)
meta_data.sort_index(inplace= True)
meta_data

,orig.ident,nCount_RNA,nFeature_RNA,percent.mt,percent.rb,log10GenesPerUMI,cell_type_high_resolution,subject,Pathologic_diagnosis_of_AD,clinical_pathological_AD
cell_id,,,,,,,,,,
AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
AAACCCAAGAAAGTCT.10.2,ROSMAP,12099,4380,2.644847,0.438053,0.891917,Exc L2-3 CBLN2 LINC02306,ROSMAP-83589,yes,AD_with_Plaques
AAACCCAAGAATAACC.15.10,ROSMAP,30621,6347,1.632866,0.372294,0.847649,Exc L2-3 CBLN2 LINC02306,ROSMAP-65083,no,NCI_with_No_Plaques
AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
AAACCCAAGAATAGTC.32.11,ROSMAP,5254,2591,0.894557,0.266464,0.917478,Exc L2-3 CBLN2 LINC02306,ROSMAP-20815,no,NCI_with_No_Plaques
...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTCGTCAC.16.11,ROSMAP,15029,4591,5.875308,0.266152,0.876698,Exc L2-3 CBLN2 LINC02306,ROSMAP-53808,yes,AD_with_Plaques
TTTGTTGTCTCGTCAC.16.11,ROSMAP,15029,4591,5.875308,0.266152,0.876698,Exc L2-3 CBLN2 LINC02306,ROSMAP-53808,yes,AD_with_Plaques
TTTGTTGTCTCTCGCA.7.4,ROSMAP,32152,6608,0.127519,0.276810,0.847547,Exc L2-3 CBLN2 LINC02306,ROSMAP-78346,yes,AD_with_Plaques


In [29]:
data = data.loc[meta_data.index,]
data

ensemble_gene_name,ENSG00000259905,ENSG00000185823,ENSG00000128739,ENSG00000273173,ENSG00000114062,ENSG00000235731,ENSG00000206190,ENSG00000206187,ENSG00000259150,ENSG00000235160,...,ENSG00000177508,ENSG00000261803,ENSG00000283304,ENSG00000245694,ENSG00000176842,ENSG00000159387,ENSG00000087245,ENSG00000260135,ENSG00000087253,ENSG00000256812
cell_id,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGAAAGTCT.10.2,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAAAGTCT.10.2,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAACC.15.10,0,0,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAGTC.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAACCCAAGAATAGTC.32.11,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGTCTCGTCAC.16.11,0,0,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTCGTCAC.16.11,0,0,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TTTGTTGTCTCTCGCA.7.4,0,0,3,0,6,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
y = meta_data.clinical_pathological_AD.tolist()

In [31]:

X = data

In [32]:
from sklearn.model_selection import train_test_split

# Step 2: Prepare the data
# Example data


# Step 3: Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Output the results
#print("X_train:", X_train)
#print("X_test:", X_test)
#print("y_train:", y_train)
#print("y_test:", y_test)

In [33]:
X_train

ensemble_gene_name,ENSG00000259905,ENSG00000185823,ENSG00000128739,ENSG00000273173,ENSG00000114062,ENSG00000235731,ENSG00000206190,ENSG00000206187,ENSG00000259150,ENSG00000235160,...,ENSG00000177508,ENSG00000261803,ENSG00000283304,ENSG00000245694,ENSG00000176842,ENSG00000159387,ENSG00000087245,ENSG00000260135,ENSG00000087253,ENSG00000256812
cell_id,,,,,,,,,,,,,,,,,,,,,
ATACTTCGTTCAATCG.5.2,0,0,1,0,2,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
GGACAAGGTACCGCTG.40.0,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TACACCCAGCCTTCTC.28.4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AGGTCCGAGGCAGTCA.18.1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAGACAACACCTGCGA.10.2,0,1,2,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GAGTCATAGGCCTGCT.33.2,0,0,2,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TGCAGGCCAACGCATT.30.6,0,0,10,0,15,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TATCTTGAGCATGCGA.10.2,0,0,3,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
pd.DataFrame(y_train)

,0
0,NCI_with_No_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,AD_with_Plaques
4,AD_with_Plaques
...,...
87779,NCI_with_No_Plaques
87780,AD_with_Plaques
87781,AD_with_Plaques
87782,AD_with_Plaques


In [35]:
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  #print(len(mkh))
  #print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  return mkh, mkl


def get_expression(rdata, qdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)

    #print(rdt)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    #print(nm)
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    #print(gid)
    qdata = qdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
    return train_x, test_x, train_y


In [36]:

r_data_tmp = X_train.T
q_data_tmp = X_test.T
r_label_tmp = pd.DataFrame(y_train).rename(columns = {0:'diagnosis'})
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.99,
                                              thrl=0.99,
                                              normalization=True,
                                              marker=True)

12
12


In [37]:
'''
import numpy as np
import pandas as pd


def fphi(x):
  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def frho(x):
  zt = (1 - (1 / (
      x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)
  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)
  return zp


def fmkg(phi, rho, thr1=0.95, thr2=0.9):
  #print(phi)
  #print(rho)
  gnm = phi.index.values
  ctpnm = phi.columns.values
  phi = np.array(phi)
  rho = np.array(rho)
  nummkg1 = round((1 - thr1) * phi.shape[0])
  nummkg2 = round((1 - thr2) * phi.shape[0])
  print(nummkg1)
  print(nummkg2)
  alpha = []
  beta = []
  for i in range(0, phi.shape[1]):
    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))
    beta = np.append(beta, np.quantile(rho[:, i], thr2))
  mkh = []
  mkl = []
  for i in range(0, phi.shape[1]):
    mkh = np.concatenate([mkh, gnm[phi[:, i] >= alpha[i]][0:nummkg1]], axis=0)
    mkl = np.concatenate([mkl, gnm[rho[:, i] >= beta[i]][0:nummkg2]], axis=0)
  
  print(len(mkh))
  print(len(mkl))
  mkh = mkh.reshape(nummkg1, phi.shape[1])
  mkl = mkl.reshape(nummkg2, rho.shape[1])
  mkh = pd.DataFrame(mkh, columns=ctpnm)
  mkl = pd.DataFrame(mkl, columns=ctpnm)
  print(mkh)
  print(mkl)
  return mkh, mkl


def get_expression(rdata, qdata, vdata, rlabel, thrh=0.95, thrl=0.9, normalization=True, marker=True):
    # calculate sum of cell type
    rulabel = rlabel.iloc[:, 0].unique()
    rdt = pd.DataFrame(data=None, columns=None)
    for l in rulabel:
      rdata_l = rdata.iloc[:, rlabel[(rlabel["diagnosis"] == l)].index.tolist()] #row value in rlabel, is column in rdata
      zs = rdata_l.apply(lambda x: x.sum(), axis=1)
      rdt = pd.concat([rdt, pd.DataFrame(data=zs, columns=[l])], axis=1)
      

    # normalization
    if normalization:
        rdt_df = rdt
        rdata_df = rdata
        qdata_df = qdata
        vdata_df = vdata
        rdt = np.array(rdt_df, dtype=np.float32)
        rdata = np.array(rdata_df, dtype=np.float32)
        qdata = np.array(qdata_df, dtype=np.float32)
        vdata = np.array(vdata_df, dtype=np.float32)
        rdt = np.divide(rdt, np.sum(rdt, axis=0, keepdims=True)) * 10000
        rdata = np.log2(np.divide(rdata, np.sum(rdata, axis=0, keepdims=True)) * 10000 + 1)
        qdata = np.log2(np.divide(qdata, np.sum(qdata, axis=0, keepdims=True)) * 10000 + 1)
        vdata = np.log2(np.divide(vdata, np.sum(vdata, axis=0, keepdims=True)) * 10000 + 1)

        rdt = pd.DataFrame(data=rdt, columns=rdt_df.columns, index=rdt_df.index)
        rdata = pd.DataFrame(data=rdata, columns=rdata_df.columns, index=rdata_df.index)
        qdata = pd.DataFrame(data=qdata, columns=qdata_df.columns, index=qdata_df.index)
        vdata = pd.DataFrame(data=vdata, columns=vdata_df.columns, index=vdata_df.index)
    #print(rdt.head())
    #print(rdata.head())
    # match gene ensembl ids between the query data and the reference data
    nm = qdata.index.tolist()
    zg = pd.DataFrame(data=nm, columns=['gene'])
    gid = []
    for i in range(0, len(nm)):
      if nm[i] not in rdata.index.tolist():
        zg.iloc[i] = ''
      else:
        gid = gid + [nm[i]]
    qdata = qdata.loc[gid, :]
    vdata = vdata.loc[gid, :]
    rdata = rdata.loc[gid, :]
    rdt = rdt.loc[gid, :]

    rlabel.index = rdata.columns

    if not marker:
        train_x = rdata
        train_y = rlabel
        test_x = qdata
        val_x = vdata
    else:
        # calculate important genes
        phi = fphi(rdt)
        rho = frho(rdt)
        mkh, mkl = fmkg(phi, rho, thrh, thrl)
        mkg = np.unique(np.append(np.array(mkh).reshape(1, -1), np.array(mkl).reshape(1, -1)))

        # return training set and testing set
        train_x = rdata.loc[mkg, :]
        train_y = rlabel
        test_x = qdata.loc[mkg, :]
        val_x = vdata.loc[mkg, :]
    return train_x, test_x, val_x, train_y
'''

'\nimport numpy as np\nimport pandas as pd\n\n\ndef fphi(x):\n  zt = (1 - (x + 1).div(x.max(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)\n  zp = (x + 1).div(x.max(axis=1) + 1, axis=0).mul(zt, axis=0)\n  return zp\n\n\ndef frho(x):\n  zt = (1 - (1 / (\n      x + 1)).mul(x.min(axis=1) + 1, axis=0)).sum(axis=1) / (x.shape[1] - 1)\n  zp = (1 / (x + 1)).mul(x.min(axis=1) + 1, axis=0).mul(zt, axis=0)\n  return zp\n\n\ndef fmkg(phi, rho, thr1=0.95, thr2=0.9):\n  #print(phi)\n  #print(rho)\n  gnm = phi.index.values\n  ctpnm = phi.columns.values\n  phi = np.array(phi)\n  rho = np.array(rho)\n  nummkg1 = round((1 - thr1) * phi.shape[0])\n  nummkg2 = round((1 - thr2) * phi.shape[0])\n  print(nummkg1)\n  print(nummkg2)\n  alpha = []\n  beta = []\n  for i in range(0, phi.shape[1]):\n    alpha = np.append(alpha, np.quantile(phi[:, i], thr1))\n    beta = np.append(beta, np.quantile(rho[:, i], thr2))\n  mkh = []\n  mkl = []\n  for i in range(0, phi.shape[1]):\n    mkh = np.concatenate([mkh, gn

In [38]:
X_train.T

cell_id,ATACTTCGTTCAATCG.5.2,GGACAAGGTACCGCTG.40.0,TACACCCAGCCTTCTC.28.4,AGGTCCGAGGCAGTCA.18.1,AAGACAACACCTGCGA.10.2,GTCGTAATCCAATGCA.13.12,TTCTACAGTTATTCTC.31.1,CCGTGAGCATGCCGGT.13.7,ATGAAAGGTATACCTG.20.11,AGATCCAGTATCTCGA.4.13,...,GGAATGGGTCGTAATC.18.8,CACAGTAAGACTAGAT.11.0,GCATGATCAGGAATCG.42.2,AATCACGGTACGCTTA.20.4,CGATGGCCATCACGAT.8.1,GAGTCATAGGCCTGCT.33.2,TGCAGGCCAACGCATT.30.6,TATCTTGAGCATGCGA.10.2,AAAGGGCAGAATCGAT.28.2,ACTTTGTAGGTCGTGA.1.4
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000259905,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0
ENSG00000185823,0,0,0,0,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
ENSG00000128739,1,4,1,1,2,14,0,4,11,1,...,0,1,0,8,0,2,10,3,0,1
ENSG00000273173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000114062,2,0,0,0,3,3,1,1,4,0,...,5,2,0,6,1,3,15,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000159387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000087245,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000260135,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [39]:
pd.DataFrame(y_train).rename(columns = {0:'diagnosis'})

,diagnosis
0,NCI_with_No_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,AD_with_Plaques
4,AD_with_Plaques
...,...
87779,NCI_with_No_Plaques
87780,AD_with_Plaques
87781,AD_with_Plaques
87782,AD_with_Plaques


In [40]:

r_data_tmp = X_train.T
q_data_tmp = X_test.T
r_label_tmp = pd.DataFrame(y_train).rename(columns = {0:'diagnosis'})
train_x, test_x, train_y = get_expression(r_data_tmp,
                                              q_data_tmp,
                                              r_label_tmp,
                                              thrh=0.80,
                                              thrl=0.80,
                                              normalization=True,
                                              marker=True)

235
235


In [41]:
train_x

cell_id,ATACTTCGTTCAATCG.5.2,GGACAAGGTACCGCTG.40.0,TACACCCAGCCTTCTC.28.4,AGGTCCGAGGCAGTCA.18.1,AAGACAACACCTGCGA.10.2,GTCGTAATCCAATGCA.13.12,TTCTACAGTTATTCTC.31.1,CCGTGAGCATGCCGGT.13.7,ATGAAAGGTATACCTG.20.11,AGATCCAGTATCTCGA.4.13,...,GGAATGGGTCGTAATC.18.8,CACAGTAAGACTAGAT.11.0,GCATGATCAGGAATCG.42.2,AATCACGGTACGCTTA.20.4,CGATGGCCATCACGAT.8.1,GAGTCATAGGCCTGCT.33.2,TGCAGGCCAACGCATT.30.6,TATCTTGAGCATGCGA.10.2,AAAGGGCAGAATCGAT.28.2,ACTTTGTAGGTCGTGA.1.4
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000005187,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
ENSG00000005189,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.733368
ENSG00000005844,0.000000,0.000000,0.00000,0.0,0.000000,4.089637,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.868434,0.0,0.000000
ENSG00000006007,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,4.473554,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.733368
ENSG00000013364,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,4.074493,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.843311,0.0,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000290203,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.688248,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
ENSG00000291606,4.113801,0.000000,4.84302,0.0,3.894296,3.171979,0.000000,0.000000,4.872264,0.000000,...,0.000000,4.918375,4.590643,2.843311,0.0,4.019764,0.000000,4.818180,0.0,3.733368
ENSG00000291629,0.000000,0.000000,0.00000,0.0,3.894296,3.171979,0.000000,4.074493,3.012950,4.693521,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.307722,0.000000,0.0,3.733368


In [42]:
train_y

,diagnosis
cell_id,
ATACTTCGTTCAATCG.5.2,NCI_with_No_Plaques
GGACAAGGTACCGCTG.40.0,AD_with_Plaques
TACACCCAGCCTTCTC.28.4,AD_with_Plaques
AGGTCCGAGGCAGTCA.18.1,AD_with_Plaques
AAGACAACACCTGCGA.10.2,AD_with_Plaques
...,...
GAGTCATAGGCCTGCT.33.2,NCI_with_No_Plaques
TGCAGGCCAACGCATT.30.6,AD_with_Plaques
TATCTTGAGCATGCGA.10.2,AD_with_Plaques


In [43]:
train_y.diagnosis.value_counts()

diagnosis
AD_with_Plaques        44003
NCI_with_No_Plaques    43781
Name: count, dtype: int64

In [44]:
test_x

cell_id,GGGATGATCCCATAAG.31.9,TATTACCAGGATATAC.5.1,AAGTTCGTCGTCTACC.7.13,ATCGCCTGTTCCACGG.9.8,ATTTACCTCAGCTGAT.23.9,TCGTGGGCATGATGCT.10.7,GGACAGACAGCTGTGC.5.1,CCCAGTTGTTCATGGT.35.1,ATGAGGGAGAAATTCG.3.9,CTTCTCTTCGACATAC.7.13,...,ATGACCATCCAACCGG.30.10,GAGACTTCATCGCTGG.17.2,GTAGTACGTATTCTCT.8.8,GTACAACGTTCCTAAG.8.7,ATGAGTCCAAGTGTCT.16.2,TGCGGGTGTTGGACTT.16.11,AAGTTCGTCAAACGAA.25.8,GATCGTATCATCGGGC.16.4,CTCTGGTTCCATCTGC.21.11,GGGAATGGTGATGTGG.11.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000005187,0.0,0.000000,0.000000,0.000000,0.000000,4.422899,0.000000,0.0,4.330591,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000005189,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,3.608598,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000005844,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000006007,0.0,5.911011,0.000000,0.000000,4.259183,0.000000,0.000000,0.0,0.000000,3.323227,...,0.000000,3.608598,0.000000,0.000000,0.000000,4.330591,4.118244,0.000000,0.0,0.0
ENSG00000013364,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000290203,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000291606,0.0,0.000000,0.000000,2.627295,5.221013,0.000000,6.006548,0.0,0.000000,0.000000,...,4.568474,4.548216,4.739143,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000291629,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000,3.608598,0.000000,3.020124,0.000000,0.000000,0.000000,3.326742,0.0,0.0


In [45]:
ensembl_pathway_relation = '../../usman/CellTICS/reactome/Ensembl2Reactome_All_Levels.txt'
species = 'human'
pathway_names = '../../usman/CellTICS/reactome/ReactomePathways.txt'
pathway_relation = '../../usman/CellTICS/reactome/ReactomePathwaysRelation.txt'
datatype = 'diagnosis'

In [46]:
pathway_genes = get_gene_pathways(ensembl_pathway_relation, species=species)

In [47]:
#set(pathway_genes.gene)-set(test_x.index)

In [48]:
pathway_genes

,group,gene
0,R-HSA-162699,ENSG00000000419
1,R-HSA-163125,ENSG00000000419
2,R-HSA-1643685,ENSG00000000419
3,R-HSA-3781865,ENSG00000000419
4,R-HSA-392499,ENSG00000000419
...,...,...
756483,R-HSA-9678110,ENSG00000680121
756484,R-HSA-9679191,ENSG00000680121
756485,R-HSA-9679506,ENSG00000680121
756486,R-HSA-9694516,ENSG00000680121


In [49]:
train_x

cell_id,ATACTTCGTTCAATCG.5.2,GGACAAGGTACCGCTG.40.0,TACACCCAGCCTTCTC.28.4,AGGTCCGAGGCAGTCA.18.1,AAGACAACACCTGCGA.10.2,GTCGTAATCCAATGCA.13.12,TTCTACAGTTATTCTC.31.1,CCGTGAGCATGCCGGT.13.7,ATGAAAGGTATACCTG.20.11,AGATCCAGTATCTCGA.4.13,...,GGAATGGGTCGTAATC.18.8,CACAGTAAGACTAGAT.11.0,GCATGATCAGGAATCG.42.2,AATCACGGTACGCTTA.20.4,CGATGGCCATCACGAT.8.1,GAGTCATAGGCCTGCT.33.2,TGCAGGCCAACGCATT.30.6,TATCTTGAGCATGCGA.10.2,AAAGGGCAGAATCGAT.28.2,ACTTTGTAGGTCGTGA.1.4
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000005187,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
ENSG00000005189,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.733368
ENSG00000005844,0.000000,0.000000,0.00000,0.0,0.000000,4.089637,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,3.868434,0.0,0.000000
ENSG00000006007,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,4.473554,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,3.733368
ENSG00000013364,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,4.074493,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.843311,0.0,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000290203,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,3.688248,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
ENSG00000291606,4.113801,0.000000,4.84302,0.0,3.894296,3.171979,0.000000,0.000000,4.872264,0.000000,...,0.000000,4.918375,4.590643,2.843311,0.0,4.019764,0.000000,4.818180,0.0,3.733368
ENSG00000291629,0.000000,0.000000,0.00000,0.0,3.894296,3.171979,0.000000,4.074493,3.012950,4.693521,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,2.307722,0.000000,0.0,3.733368


In [68]:
import numpy as np
import pandas as pd
import networkx as nx


def get_gene_pathways(input_file, species='mouse'):
    dt = pd.read_table(input_file, header=None)
    if species == 'mouse':
        ensembl = dt[(dt[0] >= 'ENSMUSG00000000000') & (dt[0] <= 'ENSMUST99999999999')]
    elif species == 'human':
        ensembl = dt[((dt[0] >= 'ENSG00000000000000') & (dt[0] <= 'ENSG00099999999999'))
                     | ((dt[0] >= 'ENSP00000000000000') & (dt[0] <= 'ENSP00099999999999'))
                     | ((dt[0] >= 'ENST00000000000000') & (dt[0] <= 'ENST00099999999999'))]
    elif species == 'rat':
        ensembl = dt[(dt[0] >= 'ENSRNOG00000000000') & (dt[0] <= 'ENSRNOT99999999999')]

    ensembl = ensembl.iloc[:, 0:2]
    ensembl.columns = ['gene', 'group']
    ensembl = pd.DataFrame(ensembl, columns=['group', 'gene'])
    ensembl.index = range(0, ensembl.shape[0])

    if species == 'mouse':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSMUSG' + ensembl.iloc[i, 1][7:18]
    elif species == 'human':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSG000' + ensembl.iloc[i, 1][7:18]
    elif species == 'rat':
        for i in range(0, ensembl.shape[0]):
            ensembl.iloc[i, 1] = 'ENSRNOG' + ensembl.iloc[i, 1][7:18]
    ensembl = ensembl.drop_duplicates()
    ensembl.index = range(0, ensembl.shape[0])
    return ensembl


class Reactome():

    def __init__(self, pathway_names, pathway_genes, relations_file_name, species):
        self.pathway_names = self.load_names(pathway_names)
        self.pathway_genes = pathway_genes
        self.hierarchy = self.load_hierarchy(relations_file_name)
        self.species = species

    def load_names(self, pathway_names):
        filename = pathway_names
        df = pd.read_csv(filename, sep='\t')
        df.columns = ['reactome_id', 'pathway_name', 'species']
        return df

    def load_hierarchy(self, relations_file_name):
        filename = relations_file_name
        df = pd.read_table(filename, header=None)
        df.columns = ['child', 'parent']
        return df


class ReactomeNetwork():

    def __init__(self, pathway_names, pathway_genes, relations_file_name, species):
        self.reactome = Reactome(pathway_names, pathway_genes, relations_file_name, species)  # low level access to reactome pathways and genes
        self.netx = self.get_reactome_networkx()

    def get_reactome_networkx(self):
        if hasattr(self, 'netx'):
            return self.netx
        hierarchy = self.reactome.hierarchy
        print(self.reactome.species)
        # filter hierarchy to have mouse pathways only
        if self.reactome.species == 'mouse':
            abbr = 'MMU'
        elif self.reactome.species == 'human':
            abbr = 'HSA'
        elif self.reactome.species == 'rat':
            abbr = 'RNO'
        print(abbr)
        species_hierarchy = hierarchy[hierarchy['child'].str.contains(abbr)]
        net = nx.from_pandas_edgelist(species_hierarchy, 'child', 'parent', create_using=nx.DiGraph())
        net.name = 'reactome'

        # add root node
        roots = [n for n, d in net.in_degree() if d == 0]
        root_node = 'root'
        edges = [(root_node, n) for n in roots]
        net.add_edges_from(edges)

        return net


def add_duplicated_edges(G, node, n_levels):
    edges = []
    source = node
    for l in range(n_levels):
        target = node + '_copy' + str(l + 1)
        edge = (source, target)
        source = target
        edges.append(edge)

    G.add_edges_from(edges)
    return G, target


def add_gene_edges(G, node, pathways, genes_df):
    genes = []
    if type(pathways) == str:
        genes = genes + list(genes_df[genes_df['group'] == pathways]['gene'])
    else:
        for i in range(len(pathways)):
            genes = genes + list(genes_df[genes_df['group'] == pathways[i]]['gene'])
    genes = list(set(genes))
    edges = []
    source = node
    for target in genes:
        edge = (source, target)
        edges.append(edge)
    G.add_edges_from(edges)
    return G


def get_nodes_at_level(net, distance):
    nodes = set(nx.ego_graph(net, 'root', radius=distance))     # get all nodes within distance around the query node
    if distance >= 1.:
        nodes -= set(nx.ego_graph(net, 'root', radius=distance - 1))     # remove nodes that are not at the specified distance but closer
    return list(nodes)


def gene_mapping(gene, df):
    inter_gene = list(set(gene) & set(df['gene']))
    genedict = {}
    genelist = [df.iloc[0, 0]]
    genedict[df.iloc[0, 1]] = genelist
    for i in range(1, df.shape[0]):
        if df.iloc[i, 1] == df.iloc[i - 1, 1]:
            genelist.append(df.iloc[i, 0])
        else:
            genedict[df.iloc[i - 1, 1]] = genelist
            genelist = [df.iloc[i, 0]]
    mappingdf = pd.DataFrame(data=None, columns=['group', 'gene'])
    for j in range(len(inter_gene)):
        mappingdf_iter = {'group': genedict[inter_gene[j]],
                  'gene': [inter_gene[j]] * len(genedict[inter_gene[j]])}
        mappingdf_iter = pd.DataFrame(mappingdf_iter)
        mappingdf = pd.concat([mappingdf, mappingdf_iter])
    return mappingdf


def get_masking(pathway_names, pathway_genes, relations_file_name, train_x, test_x, train_y, datatype, species='mouse', n_hidden=5):
    print(species)
    reactome_net = ReactomeNetwork(pathway_names, pathway_genes, relations_file_name, species)
    genes_df = reactome_net.reactome.pathway_genes
    genes_df = gene_mapping(train_x.index.tolist(), genes_df)

    original_network = reactome_net.netx
    original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]
    in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
    while in_genes_df.count(False) > 0:
        in_genes_df = [False for x in range(0, len(original_terminal_nodes))]
        for i in range(len(original_terminal_nodes)):
            if original_terminal_nodes[i] in genes_df['group'].to_list():
                in_genes_df[i] = True
        for i in range(len(original_terminal_nodes)):
            if in_genes_df[i] == False:
                original_network.remove_node(original_terminal_nodes[i])
        original_terminal_nodes = [n for n, d in original_network.out_degree() if d == 0]

    sub_graph = nx.ego_graph(original_network, 'root', radius=n_hidden) #subgraph of neighbors centered at node "root" <= a given radius (n_level).
    sub_terminal_nodes = [n for n, d in sub_graph.out_degree() if d == 0]
    for node in sub_terminal_nodes:
        distance = len(nx.shortest_path(sub_graph, source='root', target=node)) #len of distance: num of nodes in the shortest path
        if (distance == n_hidden + 1) & (node not in original_terminal_nodes):
            part_graph = nx.ego_graph(original_network, node, radius=100)
            corresponding_terminal_nodes = [n for n, d in part_graph.out_degree() if d == 0]
            sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
        elif (distance == n_hidden + 1) & (node in original_terminal_nodes):
            corresponding_terminal_nodes = node
            sub_graph = add_gene_edges(sub_graph, node, corresponding_terminal_nodes, genes_df)
        elif distance <= n_hidden:
            diff = n_hidden - distance + 1
            sub_graph, copy_node = add_duplicated_edges(sub_graph, node, diff)
            sub_graph = add_gene_edges(sub_graph, copy_node, node, genes_df)
    final_network = sub_graph

    n_level = n_hidden + 2
    layers_node = {}
    for i in range(n_level):
        nodes = get_nodes_at_level(final_network, i)
        layers_node[i] = nodes
    layers_node[0] = list(set(train_y[datatype]))

    layers_relationship = []
    for i in range(n_hidden):
        nodes = get_nodes_at_level(final_network, i + 1)
        dict = {}
        for n in nodes:
            next = list(final_network.successors(n))
            dict[n] = next
        layers_relationship.append(dict)

    masking = {}
    masking[0] = pd.DataFrame(data=1, index=layers_node[0], columns=layers_node[1])
    for i in range(1, n_level-1):
        masking[i] = pd.DataFrame(data=0, index=layers_node[i], columns=layers_node[i + 1])
        for n in layers_node[i]:
            for node in layers_relationship[i - 1][n]:
                if node in layers_node[i + 1]:
                    masking[i].loc[n, node] = 1

    gene_in_network = list(masking[n_hidden].columns)
    train_x = train_x.loc[gene_in_network, :]
    test_x = test_x.loc[gene_in_network, :]
    for i in range(len(masking) - 1):
        masking[i] = np.array(masking[i + 1])
    del masking[len(masking) - 1]
    return masking, layers_node, train_x, test_x


In [69]:
print(species)

human


In [70]:
n_hidden = 4
masking, layers_node, train_x, test_x = get_masking(pathway_names,
                                                        pathway_genes,
                                                        pathway_relation,
                                                        train_x,
                                                        test_x,
                                                        train_y,
                                                        datatype,
                                                        species,
                                                        n_hidden)

human
human
HSA


In [71]:
train_x

cell_id,ATACTTCGTTCAATCG.5.2,GGACAAGGTACCGCTG.40.0,TACACCCAGCCTTCTC.28.4,AGGTCCGAGGCAGTCA.18.1,AAGACAACACCTGCGA.10.2,GTCGTAATCCAATGCA.13.12,TTCTACAGTTATTCTC.31.1,CCGTGAGCATGCCGGT.13.7,ATGAAAGGTATACCTG.20.11,AGATCCAGTATCTCGA.4.13,...,GGAATGGGTCGTAATC.18.8,CACAGTAAGACTAGAT.11.0,GCATGATCAGGAATCG.42.2,AATCACGGTACGCTTA.20.4,CGATGGCCATCACGAT.8.1,GAGTCATAGGCCTGCT.33.2,TGCAGGCCAACGCATT.30.6,TATCTTGAGCATGCGA.10.2,AAAGGGCAGAATCGAT.28.2,ACTTTGTAGGTCGTGA.1.4
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000137845,5.642114,0.000000,4.843020,0.0,0.000000,4.089637,6.219738,0.000000,4.473554,0.0,...,2.796038,0.000000,0.000000,3.739125,5.658353,0.000000,3.684036,3.868434,0.000000,5.649630
ENSG00000138604,5.071524,5.307794,5.817666,0.0,4.844951,3.171979,0.000000,4.074493,3.012950,0.0,...,5.917743,4.918375,0.000000,0.000000,0.000000,0.000000,4.626792,0.000000,0.000000,0.000000
ENSG00000169752,4.113801,0.000000,0.000000,0.0,4.844951,0.000000,0.000000,0.000000,4.473554,0.0,...,0.000000,0.000000,0.000000,4.684067,0.000000,4.019764,3.154132,0.000000,0.000000,0.000000
ENSG00000137857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,2.843311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000035664,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000069943,0.000000,0.000000,0.000000,0.0,3.894296,0.000000,0.000000,0.000000,3.920700,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.307722,0.000000,0.000000,0.000000
ENSG00000013364,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,4.074493,0.000000,0.0,...,0.000000,0.000000,0.000000,2.843311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000182054,0.000000,5.307794,0.000000,0.0,0.000000,3.171979,0.000000,4.074493,0.000000,0.0,...,0.000000,0.000000,4.590643,3.739125,0.000000,0.000000,0.000000,4.818180,0.000000,0.000000


In [72]:
train_y

,diagnosis
cell_id,
ATACTTCGTTCAATCG.5.2,NCI_with_No_Plaques
GGACAAGGTACCGCTG.40.0,AD_with_Plaques
TACACCCAGCCTTCTC.28.4,AD_with_Plaques
AGGTCCGAGGCAGTCA.18.1,AD_with_Plaques
AAGACAACACCTGCGA.10.2,AD_with_Plaques
...,...
GAGTCATAGGCCTGCT.33.2,NCI_with_No_Plaques
TGCAGGCCAACGCATT.30.6,AD_with_Plaques
TATCTTGAGCATGCGA.10.2,AD_with_Plaques


In [73]:
test_x

cell_id,GGGATGATCCCATAAG.31.9,TATTACCAGGATATAC.5.1,AAGTTCGTCGTCTACC.7.13,ATCGCCTGTTCCACGG.9.8,ATTTACCTCAGCTGAT.23.9,TCGTGGGCATGATGCT.10.7,GGACAGACAGCTGTGC.5.1,CCCAGTTGTTCATGGT.35.1,ATGAGGGAGAAATTCG.3.9,CTTCTCTTCGACATAC.7.13,...,ATGACCATCCAACCGG.30.10,GAGACTTCATCGCTGG.17.2,GTAGTACGTATTCTCT.8.8,GTACAACGTTCCTAAG.8.7,ATGAGTCCAAGTGTCT.16.2,TGCGGGTGTTGGACTT.16.11,AAGTTCGTCAAACGAA.25.8,GATCGTATCATCGGGC.16.4,CTCTGGTTCCATCTGC.21.11,GGGAATGGTGATGTGG.11.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000137845,0.000000,0.000000,4.186794,5.406990,5.221013,4.422899,0.0,6.434129,5.866942,5.210858,...,4.568474,5.517050,0.000000,4.481371,3.663187,4.330591,5.646734,3.326742,0.0,0.0
ENSG00000138604,0.000000,0.000000,5.717954,5.003242,4.259183,0.000000,0.0,0.000000,4.330591,3.323227,...,0.000000,5.112476,4.739143,3.020124,0.000000,0.000000,0.000000,5.214657,0.0,0.0
ENSG00000169752,5.011227,0.000000,0.000000,0.000000,3.332623,0.000000,0.0,0.000000,4.330591,0.000000,...,0.000000,0.000000,4.739143,3.020124,4.605093,0.000000,5.076098,3.326742,0.0,0.0
ENSG00000137857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,3.628031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000035664,0.000000,0.000000,0.000000,0.000000,0.000000,4.422899,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000069943,0.000000,0.000000,0.000000,3.505552,3.332623,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.020124,5.170159,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000013364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000182054,0.000000,0.000000,0.000000,3.505552,3.332623,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,3.608598,0.000000,0.000000,0.000000,0.000000,0.000000,4.252995,0.0,0.0


In [74]:
#pd.DataFrame(test_x.index).to_csv('../../preprocessed_data/inhibitory_neuron/marker_genes.csv', index=False)

In [75]:
masking

{0: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 1: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 2: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 3: array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])}

In [76]:
len(masking)

4

In [77]:
len(layers_node[0])

2

In [78]:
masking[0].shape, len(layers_node[1])

((23, 82), 23)

In [ ]:
masking[1].shape, len(layers_node[2])

In [ ]:
#masking[2].shape, len(layers_node[3])

In [ ]:
 #len(layers_node[4])

In [ ]:
#sum(masking[2].flatten())

In [ ]:
#sum(masking[1].flatten())

In [ ]:
#sum(masking[0].flatten())

In [ ]:
#masking[2].shape

In [79]:
diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
train_y['diagnosis_binary'] = train_y['diagnosis'].map(diagnosis_mapping)

# Display the first few rows to verify the mapping
print(train_y.head())
train_y.drop(columns=['diagnosis'], inplace=True)
train_y

                                 diagnosis  diagnosis_binary
cell_id                                                     
ATACTTCGTTCAATCG.5.2   NCI_with_No_Plaques                 0
GGACAAGGTACCGCTG.40.0      AD_with_Plaques                 1
TACACCCAGCCTTCTC.28.4      AD_with_Plaques                 1
AGGTCCGAGGCAGTCA.18.1      AD_with_Plaques                 1
AAGACAACACCTGCGA.10.2      AD_with_Plaques                 1


,diagnosis_binary
cell_id,
ATACTTCGTTCAATCG.5.2,0
GGACAAGGTACCGCTG.40.0,1
TACACCCAGCCTTCTC.28.4,1
AGGTCCGAGGCAGTCA.18.1,1
AAGACAACACCTGCGA.10.2,1
...,...
GAGTCATAGGCCTGCT.33.2,0
TGCAGGCCAACGCATT.30.6,1
TATCTTGAGCATGCGA.10.2,1


In [80]:
y_test = pd.DataFrame(y_test).rename(columns={0:'diagnosis'})
y_test

,diagnosis
0,NCI_with_No_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,NCI_with_No_Plaques
4,NCI_with_No_Plaques
...,...
29257,AD_with_Plaques
29258,NCI_with_No_Plaques
29259,AD_with_Plaques
29260,NCI_with_No_Plaques


In [81]:
train_y.diagnosis_binary.value_counts()

diagnosis_binary
1    44003
0    43781
Name: count, dtype: int64

In [82]:
try:
    diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
    t['diagnosis_binary'] = qlabel['diagnosis'].map(diagnosis_mapping)
    
    # Display the first few rows to verify the mapping
    print(qlabel.head())
    qlabel.drop(columns=['diagnosis'], inplace=True)
    qlabel

except:
    print('Previously Run Before')

Previously Run Before


In [83]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class TabularDataset(Dataset):
    def __init__(self, count_matrix, label):
        # Read the CSV file
        self.data = count_matrix
        # Separate features and target
        self.features = self.data.values
        self.target = label.values
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Get features and target for a given index
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.target[idx], dtype=torch.float32)
        return features, target


In [84]:

torch.tensor(train_y.values, dtype=torch.float32)


tensor([[0.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])

In [85]:
y_test.diagnosis.value_counts()

diagnosis
NCI_with_No_Plaques    14742
AD_with_Plaques        14520
Name: count, dtype: int64

In [86]:
train_x

cell_id,ATACTTCGTTCAATCG.5.2,GGACAAGGTACCGCTG.40.0,TACACCCAGCCTTCTC.28.4,AGGTCCGAGGCAGTCA.18.1,AAGACAACACCTGCGA.10.2,GTCGTAATCCAATGCA.13.12,TTCTACAGTTATTCTC.31.1,CCGTGAGCATGCCGGT.13.7,ATGAAAGGTATACCTG.20.11,AGATCCAGTATCTCGA.4.13,...,GGAATGGGTCGTAATC.18.8,CACAGTAAGACTAGAT.11.0,GCATGATCAGGAATCG.42.2,AATCACGGTACGCTTA.20.4,CGATGGCCATCACGAT.8.1,GAGTCATAGGCCTGCT.33.2,TGCAGGCCAACGCATT.30.6,TATCTTGAGCATGCGA.10.2,AAAGGGCAGAATCGAT.28.2,ACTTTGTAGGTCGTGA.1.4
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000137845,5.642114,0.000000,4.843020,0.0,0.000000,4.089637,6.219738,0.000000,4.473554,0.0,...,2.796038,0.000000,0.000000,3.739125,5.658353,0.000000,3.684036,3.868434,0.000000,5.649630
ENSG00000138604,5.071524,5.307794,5.817666,0.0,4.844951,3.171979,0.000000,4.074493,3.012950,0.0,...,5.917743,4.918375,0.000000,0.000000,0.000000,0.000000,4.626792,0.000000,0.000000,0.000000
ENSG00000169752,4.113801,0.000000,0.000000,0.0,4.844951,0.000000,0.000000,0.000000,4.473554,0.0,...,0.000000,0.000000,0.000000,4.684067,0.000000,4.019764,3.154132,0.000000,0.000000,0.000000
ENSG00000137857,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,2.843311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000035664,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000069943,0.000000,0.000000,0.000000,0.0,3.894296,0.000000,0.000000,0.000000,3.920700,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.307722,0.000000,0.000000,0.000000
ENSG00000013364,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,4.074493,0.000000,0.0,...,0.000000,0.000000,0.000000,2.843311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000182054,0.000000,5.307794,0.000000,0.0,0.000000,3.171979,0.000000,4.074493,0.000000,0.0,...,0.000000,0.000000,4.590643,3.739125,0.000000,0.000000,0.000000,4.818180,0.000000,0.000000


In [87]:
#val_x = train_x.T.iloc[40000:,]
#val_y = train_y.iloc[40000:,]
#train_x = train_x.T.iloc[0:40000,]
#train_y = train_y.iloc[0:40000,]


In [88]:
test_x

cell_id,GGGATGATCCCATAAG.31.9,TATTACCAGGATATAC.5.1,AAGTTCGTCGTCTACC.7.13,ATCGCCTGTTCCACGG.9.8,ATTTACCTCAGCTGAT.23.9,TCGTGGGCATGATGCT.10.7,GGACAGACAGCTGTGC.5.1,CCCAGTTGTTCATGGT.35.1,ATGAGGGAGAAATTCG.3.9,CTTCTCTTCGACATAC.7.13,...,ATGACCATCCAACCGG.30.10,GAGACTTCATCGCTGG.17.2,GTAGTACGTATTCTCT.8.8,GTACAACGTTCCTAAG.8.7,ATGAGTCCAAGTGTCT.16.2,TGCGGGTGTTGGACTT.16.11,AAGTTCGTCAAACGAA.25.8,GATCGTATCATCGGGC.16.4,CTCTGGTTCCATCTGC.21.11,GGGAATGGTGATGTGG.11.1
ensemble_gene_name,,,,,,,,,,,,,,,,,,,,,
ENSG00000137845,0.000000,0.000000,4.186794,5.406990,5.221013,4.422899,0.0,6.434129,5.866942,5.210858,...,4.568474,5.517050,0.000000,4.481371,3.663187,4.330591,5.646734,3.326742,0.0,0.0
ENSG00000138604,0.000000,0.000000,5.717954,5.003242,4.259183,0.000000,0.0,0.000000,4.330591,3.323227,...,0.000000,5.112476,4.739143,3.020124,0.000000,0.000000,0.000000,5.214657,0.0,0.0
ENSG00000169752,5.011227,0.000000,0.000000,0.000000,3.332623,0.000000,0.0,0.000000,4.330591,0.000000,...,0.000000,0.000000,4.739143,3.020124,4.605093,0.000000,5.076098,3.326742,0.0,0.0
ENSG00000137857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,3.628031,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000035664,0.000000,0.000000,0.000000,0.000000,0.000000,4.422899,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000069943,0.000000,0.000000,0.000000,3.505552,3.332623,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.020124,5.170159,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000013364,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
ENSG00000182054,0.000000,0.000000,0.000000,3.505552,3.332623,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,3.608598,0.000000,0.000000,0.000000,0.000000,0.000000,4.252995,0.0,0.0


In [89]:
y_test

,diagnosis
0,NCI_with_No_Plaques
1,AD_with_Plaques
2,AD_with_Plaques
3,NCI_with_No_Plaques
4,NCI_with_No_Plaques
...,...
29257,AD_with_Plaques
29258,NCI_with_No_Plaques
29259,AD_with_Plaques
29260,NCI_with_No_Plaques


In [90]:


try:
    diagnosis_mapping = {'AD_with_Plaques': 1, 'NCI_with_No_Plaques': 0}
    y_test['diagnosis_binary'] = y_test['diagnosis'].map(diagnosis_mapping)
    
    # Display the first few rows to verify the mapping
    print(y_test.head())
    y_test.drop(columns=['diagnosis'], inplace=True)
    y_test

except:
    print('Previously Run Before')

             diagnosis  diagnosis_binary
0  NCI_with_No_Plaques                 0
1      AD_with_Plaques                 1
2      AD_with_Plaques                 1
3  NCI_with_No_Plaques                 0
4  NCI_with_No_Plaques                 0


In [91]:
y_test

,diagnosis_binary
0,0
1,1
2,1
3,0
4,0
...,...
29257,1
29258,0
29259,1
29260,0


In [92]:
train_dataset = TabularDataset(train_x.T,train_y)
val_dataset = TabularDataset(test_x.T,y_test)
#test_dataset = TabularDataset(test_x.T,qlabel)

In [93]:
y_test.value_counts()

diagnosis_binary
0                   14742
1                   14520
Name: count, dtype: int64

In [94]:
train_y.value_counts()

diagnosis_binary
1                   44003
0                   43781
Name: count, dtype: int64

In [95]:
print_information = True
num_epochs = 100
learning_rate = 0.01
batch_size = 256
l2_regularization = 0.00001
print_cost = True

In [96]:
def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    predicted_list = []
    labels_list = []
    
    with torch.no_grad():  # No need to compute gradients during evaluation
        for features, labels in dataloader:
            outputs = model(features)
            #print(outputs)
            predicted = torch.round(torch.sigmoid(outputs.data))
            #print(outputs)
            #print(predicted)
            #_, predicted = torch.sigmoid(outputs.data)
            predicted_list.append(predicted)
            labels_list.append(labels)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    #print(total)
    accuracy = 100 * correct / total
    return accuracy, predicted_list, labels_list

In [97]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


#for output_layer in range(2, len(masking) + 2):       -->n
class CustomNetwork(nn.Module):
    def __init__(self, layers_node, n, masking):
        super(CustomNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.masks = []
        self.gamma = 0.0001

        for i in range(n-1):
            #print(-1-i)
            #print(layers_node[-1-i])
            in_features = len(layers_node[-1-i])
            out_features = len(layers_node[-1-i-1])
            self.layers.append(nn.Linear(in_features, out_features, bias=False))
            self.masks.append(masking[-1-i])

        self.layers.append(nn.Linear(len(layers_node[-n]), len(layers_node[0])-1))
        self.masks = [torch.tensor(mask, dtype=torch.float32) for mask in self.masks]


    def forward(self, x):
        for i, layer in enumerate(self.layers[:-1]):
            mask = self.masks[i]
            #print(mask[0].shape)
            layer.weight.data *= mask
            #print(layer(x).shape)
            x = layer(x)
            
            '''
            x = x.unsqueeze(2)
            print('x shape before multi: ',x.shape)
            print('shape of mask: ', mask.shape )
            x =  x*mask
            x = x.squeeze(2)
            x = torch.tanh(x)
            '''
            x = torch.relu(x)
            #print('x shape: ',x.shape)
        #print(x.shape)
        
        x = self.layers[-1](x)
        return x



In [98]:
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

def model(train_dataloader , val_dataloader, layers_node, masking, output_layer, learning_rate=0.001, num_epochs=50, weight_decay = 0):

    model = CustomNetwork(layers_node, output_layer, masking)
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate,weight_decay = weight_decay )  # Using SGD with momentum
    criterion = nn.BCEWithLogitsLoss()
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    patience = 20
    best_val_accuracy = 0.0
    epochs_no_improve = 0
    early_stop = False
    
    for epoch in tqdm(range(num_epochs)):
        if early_stop:
            print("Early stopping")
            break
        epoch_cost = 0.
        
        for batch_features,batch_targets in train_dataloader:
            outputs = model(batch_features)
            #print(outputs)
            #print(batch_targets)
            #print(outputs)
            loss = criterion(outputs, batch_targets)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        
        train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
        val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
        #scheduler.step(val_accuracy)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Train_accuracy: {train_accuracy}, Val_accuracy: {val_accuracy}')

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_no_improve = 0
        # Save the best model
            torch.save(model.state_dict(), f'best_model_{output_layer}.pth')
            print('Model saved.')
        else:
            epochs_no_improve += 1
    
        # Early stopping
        '''if epochs_no_improve >= patience:
            early_stop = True
            print("Early stopping triggered")'''
        
    
    train_accuracy, predicted_list_train, labels_list_train = evaluate(model, train_dataloader)
    val_accuracy, predicted_list_val, labels_list_val = evaluate(model, val_dataloader)
    
    output_train = (predicted_list_train, labels_list_train)
    output_val = (predicted_list_val, labels_list_val)
    
    return output_train, output_val,model

In [100]:

#from neural_network import *
print_information = True
num_epochs = 5
learning_rate = 0.001
batch_size = 4096
l2_regularization = 0.0001
print_cost = True


try:
    masking = list(masking.values())
    layers_node = list(layers_node.values())
except:
    print('already_done')
# Create the DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
#test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle= True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle= True)
# Example of iterating through the DataLoader


pred_y_df = pd.DataFrame(data=0, index=test_x.columns, columns=list(range(2, len(masking) + 2)))
train_y_df = pd.DataFrame(data=0, index=train_x.columns, columns=list(range(2, len(masking) + 2)))
#masking = list(masking.values())
model_dict = dict()
#layers_node = list(layers_node.values())
activation_output = {}
for output_layer in range(2, len(masking) + 2):
    if print_information:
        print("Current sub-neural network has " + str(output_layer - 1) + " hidden layers.")
    output_train, output_test,model_dict[output_layer] = model(train_dataloader,
                                          val_dataloader,
                                          layers_node,
                                          masking,
                                          output_layer,
                                          learning_rate=learning_rate,num_epochs=num_epochs,weight_decay = l2_regularization
                                          )
    break

    

already_done
Current sub-neural network has 1 hidden layers.


 20%|███████████████████████████                                                                                                            | 1/5 [00:01<00:06,  1.67s/it]

Epoch [1/5], Loss: 0.6833, Train_accuracy: 56.902168960174976, Val_accuracy: 56.6741849497642
Model saved.


 40%|██████████████████████████████████████████████████████                                                                                 | 2/5 [00:03<00:05,  1.76s/it]

Epoch [2/5], Loss: 0.6717, Train_accuracy: 58.81481819010298, Val_accuracy: 58.46148588613219
Model saved.


...

Epoch [3/5], Loss: 0.6752, Train_accuracy: 59.73981591178347, Val_accuracy: 59.18939238602966
Model saved.


 80%|████████████████████████████████████████████████████████████████████████████████████████████████████████████                           | 4/5 [00:06<00:01,  1.76s/it]

Epoch [4/5], Loss: 0.6613, Train_accuracy: 60.45748655791488, Val_accuracy: 59.859203061991664
Model saved.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:08<00:00,  1.76s/it]

Epoch [5/5], Loss: 0.6636, Train_accuracy: 60.900619702907136, Val_accuracy: 60.30004784361971
Model saved.


In [103]:
len(output_train)

2

In [111]:
output_train[0][-1].shape

torch.Size([1768, 1])

# Fully Connected